In [1]:
#!/usr/bin/python

# import sys
import pickle
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import math
# from scipy import stats
from functools import partial

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.feature_selection import SelectPercentile, SelectFromModel, \
        f_classif, mutual_info_classif, chi2, SelectFpr, SelectFdr, RFECV
from sklearn.decomposition import FastICA, IncrementalPCA, KernelPCA, PCA, \
                                  TruncatedSVD
from sklearn.pipeline import Pipeline

# from sklearn.linear_model import LinearRegression
# from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

from sklearn.model_selection import GridSearchCV
# from genetic_selection import GeneticSelectionCV
# from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

# from time import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# sys.path.append("../tools/")
# from feature_format import featureFormat, targetFeatureSplit
# from dos2unix import crlf_to_lf
# from tester import dump_classifier_and_data

In [2]:
### Load data sets.
with open(file='../data/y_train.pkl', mode='rb') as file:
    y_train = pickle.load(file=file)
y_test = None
    
X_train_scaled_imp0 = None
with open('../data/X_train_scaled_imp0.pkl', 'rb') as file:
    X_train_scaled_imp0 = pickle.load(file=file)
X_train_scaled_imp_med = None
with open('../data/X_train_scaled_imp_med.pkl', 'rb') as file:
    X_train_scaled_imp_med = pickle.load(file=file)
X_train_scaled_imp_mv = None
with open('../data/X_train_scaled_imp_mv.pkl', 'rb') as file:
    X_train_scaled_imp_mv = pickle.load(file=file)

In [3]:
# X_train_scaled_imp0[(np.isnan(X_train_scaled_imp0)).any(axis=1)]
# X_train_scaled_imp0[(np.isinf(X_train_scaled_imp0)).any(axis=1)]
# X_train_scaled_imp0.head()

# X_train_scaled_imp_med[(np.isnan(X_train_scaled_imp_med)).any(axis=1)]
# X_train_scaled_imp_med[(np.isinf(X_train_scaled_imp_med)).any(axis=1)]
# X_train_scaled_imp_med.head()

# X_train_scaled_imp_mv[(np.isnan(X_train_scaled_imp_mv)).any(axis=1)]
# X_train_scaled_imp_mv[(np.isinf(X_train_scaled_imp_mv)).any(axis=1)]
# X_train_scaled_imp_mv.head()

In [4]:
# X_train_scaled_imp0.columns[340:]
# X_train_scaled_imp_med.columns[340:]
# X_train_scaled_imp_mv.columns[340:]

In [5]:
np.any(np.isnan(X_train_scaled_imp0))
np.all(np.isfinite(X_train_scaled_imp0))
X_train_scaled_imp0.info()

np.any(np.isnan(X_train_scaled_imp_med))
np.all(np.isfinite(X_train_scaled_imp_med))
X_train_scaled_imp_med.info()

np.any(np.isnan(X_train_scaled_imp_mv))
np.all(np.isfinite(X_train_scaled_imp_mv))
X_train_scaled_imp_mv.info()

False

True

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 336 entries, salary to k_means_10_n_9
dtypes: bool(148), float64(188)
memory usage: 139.4+ KB


False

True

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 336 entries, salary to k_means_10_n_9
dtypes: bool(148), float64(188)
memory usage: 139.4+ KB


False

True

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 336 entries, salary to k_means_10_n_9
dtypes: bool(148), float64(188)
memory usage: 139.4+ KB


In [6]:
# float64_feats = list(X_train_scaled_imp0.dtypes[X_train_scaled_imp0.dtypes \
#                                                 == np.float64].index)
# float64_feats

# X_train_scaled_imp0[float64_feats] = X_train_scaled_imp0[float64_feats]\
#                                         .astype(np.float32)
# X_train_scaled_imp0[float64_feats].dtypes


# X_train_scaled_imp_med[float64_feats] = X_train_scaled_imp_med[float64_feats]\
#                                         .astype(np.float32)
# X_train_scaled_imp_med[float64_feats].dtypes


# X_train_scaled_imp_mv[float64_feats] = X_train_scaled_imp_mv[float64_feats]\
#                                         .astype(np.float32)
# X_train_scaled_imp_mv[float64_feats].dtypes



# X_train_scaled_imp0 = X_train_scaled_imp0.astype(np.float32)
# X_train_scaled_imp0.info()

# X_train_scaled_imp_med = X_train_scaled_imp_med.astype(np.float32)
# X_train_scaled_imp_med.info()

# X_train_scaled_imp_mv = X_train_scaled_imp_mv.astype(np.float32)
# X_train_scaled_imp_mv.info()

In [7]:
### Search 'em all

n_jobs = -1

mutual_info_classif_partial = partial(mutual_info_classif, random_state=42)
DecisionTreeClassifier_partial = \
    partial(DecisionTreeClassifier, random_state=42)
RandomForestClassifier_partial = \
    partial(RandomForestClassifier, random_state=42, n_jobs=n_jobs)
AdaBoostClassifier_partial = partial(AdaBoostClassifier, random_state=42)
svm_SVC_partial = partial(svm.SVC, random_state=42)
KNeighborsClassifier_partial = partial(KNeighborsClassifier, n_jobs=n_jobs)

selectors = {
    'sel_per': {
        'sel': SelectPercentile(),
        'params': {
            'sel_per__score_func': [f_classif, chi2,
                                    mutual_info_classif_partial],
            'sel_per__percentile': [5, 10, 20, 50, 100]
        }
    }
    ###TypeError: get_params() missing 1 required positional argument: 'self'
    ### Must select estimator to begin with. Try setting, but keeping params.
#     'sel_mod': {
#         'sel': SelectFromModel(estimator=DecisionTreeClassifier),
#         'params': {
#             'sel_mod__estimator': [
#                 DecisionTreeClassifier_partial(),
#                 RandomForestClassifier_partial(),
#                 AdaBoostClassifier_partial(),
#                 svm_SVC_partial()
#             ],
#             'sel_mod__threshold': ['mean', 'median', '1.5*median']
#         }
#     },
    ### This one keeps failing with
    ### TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.
    ### Setting n_jobs to 7 (1 less than max for me) didn't work.
    ### Letting it go.
    #     ### Must select estimator to begin with. Try setting, but keeping params.
#     'RFECV': {
#         'sel': RFECV(estimator=DecisionTreeClassifier_partial()),
#         'params': {
#             'RFECV__estimator': [
#                 DecisionTreeClassifier_partial(),
#                 RandomForestClassifier_partial(),
#                 AdaBoostClassifier_partial(),
#                 svm_SVC_partial()
#             ],
#             'RFECV__n_jobs': [n_jobs]
#         }
#     }
}

decomps = {
    'fica': {
        'dec': FastICA(),
        'params': {
            'fica__algorithm': ['parallel', 'deflation'],
            'fica__fun': ['logcosh', 'exp', 'cube'],
            'fica__random_state': [42]
        }
    },
    'ipca': {
        'dec': IncrementalPCA(),
        'params': {
            ### defaults
        }
    }
#     'kpca': {
#         'dec': KernelPCA(),
#         'params': {
#             'kpca__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'cosine',
#                              'precomputed'],
#             'kpca__random_state': [42],
#             'kpca__n_jobs': [n_jobs]
#         }
#     },
    ### PCA kept throwing an error that the data contained nans, inf,
    ### or too large dtypes, despite no nans, infs, or wrong types per
    ### replications of the sklearn (and numpy) condition checks that threw the
    ### errors ("errors" because using PCA threw the error from sklearn script,
    ### but further testing showed that PCA's get_precision method may have
    ### thrown it from a NumPy script (see cells below for tracing), and that
    ### did not include dtype size).
    ### Maybe (probably?) a problem with the transformed data handed off from
    ### SelectPercentile, but I'm done testing. Just need to finish. Skip PCA.
#     'pca': {
#         'dec': PCA(),
#         'params': {
#             'pca__random_state': [42]
#         }
#     },
#     'tsvd': {
#         'dec': TruncatedSVD(),
#         'params': {
#             'tsvd__n_components': [2, 4, 8, 16, 32, 64, 128],
#             'tsvd__algorithm': ['arpack', 'randomized'],
#             'tsvd__random_state': [42]
#         }
#     }
}

classifiers = {
    'dt_clf': {
        'clf': DecisionTreeClassifier(),
        'params': {
            'dt_clf__random_state': [42]
        }
    },
    'rf_clf': {
        'clf': RandomForestClassifier(),
        'params': {
            'rf_clf__n_estimators': [4, 8, 16],
            'rf_clf__max_features': ['sqrt', 'log2', 0.75, None],
            'rf_clf__max_depth': [32, 128, None],
            'rf_clf__min_samples_split': [2, 4, 8],
            'rf_clf__min_samples_leaf': [1, 2, 4],
            'rf_clf__bootstrap': [True, False],
            'rf_clf__random_state': [42],
            'rf_clf__n_jobs': [n_jobs]
        }
    },
    'ab_clf': {
        'clf': AdaBoostClassifier(),
        'params': {
            'ab_clf__base_estimator': [
                DecisionTreeClassifier_partial(),
                RandomForestClassifier_partial(),
                AdaBoostClassifier_partial(),
                svm_SVC_partial(),
                KNeighborsClassifier_partial(),
                GaussianNB()
            ],
            'ab_clf__n_estimators': [32, 50, 64],
            'ab_clf__algorithm': ['SAMME', 'SAMME.R'],
            'ab_clf__random_state': [42]
          }
    },
    'kn_clf': {
        'clf': KNeighborsClassifier(),
        'params': {
            'kn_clf__n_neighbors': [2, 4, 8],
            'kn_clf__weights': ['uniform', 'distance'],
            'kn_clf__algorithm': ['ball_tree', 'kd_tree', 'brute'],
            'kn_clf__leaf_size': [16, 32, 64],
            'kn_clf__n_jobs': [n_jobs]
        }
    },
    'gnb_clf': {
        'clf': GaussianNB(),
        'params': {
            ### Defaults only
        }
    },
    ### SVC is taking waaay too long, performs terribly in baselines anyway.
#     'svc_clf': {
#         'clf': svm.SVC(),
#         'params': {
#             'svc_clf__C': np.exp2([-3, -1, 0, 1, 3]),
#             'svc_clf__kernel': ['linear', 'poly', 'rbf', 'sigmoid',
#                                 'precomputed'],
#             'svc_clf__degree': [2, 3, 4],
#             'svc_clf__gamma': np.exp2([-3, -1, 0, 1, 3]),
#             'svc_clf__shrinking': [True, False],
#             'svc_clf__random_state': [42]
#         }
#     }
}

# from genetic_selection import GeneticSelectionCV

def search_em_all(X_train, y_train=y_train, selectors=selectors,
                  decomps=decomps, classifiers=classifiers, pipe_verbose=True,
                  scoring='recall_weighted', n_jobs=n_jobs):
    ### Try also 'jaccard_weighted'
    gscv_dict = {}
    
    i = 0
    for selector in selectors:
        for decomp in decomps:
            for classifier in classifiers:
                
                pipe = Pipeline(steps=[
                    (selector, selectors[selector]['sel']),
                    (decomp, decomps[decomp]['dec']),
                    (classifier, classifiers[classifier]['clf'])
                ], verbose=pipe_verbose)
                
                params = {}
                params.update(selectors[selector]['params'])
                params.update(decomps[decomp]['params'])
                params.update(classifiers[classifier]['params'])
                
                params_name = selector + '_' + decomp + '_' + classifier
                print('\n', i, params_name, '\n')
                
                gscv = GridSearchCV(estimator=pipe, param_grid=params,
                                    scoring=scoring, n_jobs=n_jobs, verbose=3)
#                 gscv = GeneticSelectionCV(estimator,
#                                   cv=5,
#                                   verbose=3,
#                                   scoring=scoring,
#                                   max_features=5,
#                                   n_population=50,
#                                   crossover_proba=0.5,
#                                   mutation_proba=0.2,
#                                   n_generations=40,
#                                   crossover_independent_proba=0.5,
#                                   mutation_independent_proba=0.05,
#                                   tournament_size=3,
#                                   n_gen_no_change=10,
#                                   caching=False,
#                                   n_jobs=-1)
                gscv_dict[params_name] = gscv.fit(X=X_train, y=y_train)
                print('\n', gscv_dict[params_name])
                print('\nbest_score_:', gscv_dict[params_name].best_score_)
                print('\nbest_params_:', gscv_dict[params_name].best_params_)
                
                i += 1
                
    return gscv_dict

imp_gscvs_dict = {}
print('\nimp0\n')
imp_gscvs_dict['imp0'] = search_em_all(X_train=X_train_scaled_imp0)
print('\nimp_med\n')
imp_gscvs_dict['imp_med'] = search_em_all(X_train=X_train_scaled_imp_med)
print('\nimp_mv\n')
imp_gscvs_dict['imp_mv'] = search_em_all(X_train=X_train_scaled_imp_mv)


imp0


 0 sel_per_fica_dt_clf 

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   31.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[Pipeline] ........... (step 1 of 3) Processing sel_per, total=   0.0s
[Pipeline] .............. (step 2 of 3) Processing fica, total=   0.0s
[Pipeline] ............ (step 3 of 3) Processing dt_clf, total=   0.0s

 GridSearchCV(estimator=Pipeline(steps=[('sel_per', SelectPercentile()),
                                       ('fica', FastICA()),
                                       ('dt_clf', DecisionTreeClassifier())],
                                verbose=True),
             n_jobs=-1,
             param_grid={'dt_clf__random_state': [42],
                         'fica__algorithm': ['parallel', 'deflation'],
                         'fica__fun': ['logcosh', 'exp', 'cube'],
                         'fica__random_state': [42],
                         'sel_per__percentile': [5, 10, 20, 50, 100],
                         'sel_per__score_func': [<function f_classif at 0x000002165103D280>,
                                                 <function chi2 at 0x000002165103D5E0>,
        

[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 536 tasks      | elapsed:   35.8s
[Parallel(n_jobs=-1)]: Done 824 tasks      | elapsed:   57.0s
[Parallel(n_jobs=-1)]: Done 1176 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1592 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2072 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 2616 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 3224 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 3896 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 4632 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 5432 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 6296 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 7224 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 8216 tasks      | elapsed:  9.2m

[Pipeline] ........... (step 1 of 3) Processing sel_per, total=   0.0s


...

[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 408 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 920 tasks      | elapsed:   38.6s
[Parallel(n_jobs=-1)]: Done 1272 tasks      | elapsed:   53.8s
[Parallel(n_jobs=-1)]: Done 1688 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2168 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 2712 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 3320 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 3992 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 4050 out of 4050 | elapsed:  2.9min finished


[Pipeline] ........... (step 1 of 3) Processing sel_per, total=   0.6s
[Pipeline] .............. (step 2 of 3) Processing ipca, total=   0.0s
[Pipeline] ............ (step 3 of 3) Processing kn_clf, total=   0.0s

 GridSearchCV(estimator=Pipeline(steps=[('sel_per', SelectPercentile()),
                                       ('ipca', IncrementalPCA()),
                                       ('kn_clf', KNeighborsClassifier())],
                                verbose=True),
             n_jobs=-1,
             param_grid={'kn_clf__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'kn_clf__leaf_size': [16, 32, 64],
                         'kn_clf__n_jobs': [-1],
                         'kn_clf__n_neighbors': [2, 4, 8],
                         'kn_clf__weights': ['uniform', 'distance'],
                         'sel_per__percentile': [5, 10, 20, 50, 100],
                         'sel_per__score_func': [<function f_classif at 0x000002165103D280>,
                  

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  60 out of  75 | elapsed:    2.1s remaining:    0.4s


[Pipeline] ........... (step 1 of 3) Processing sel_per, total=   0.0s
[Pipeline] .............. (step 2 of 3) Processing ipca, total=   0.0s
[Pipeline] ........... (step 3 of 3) Processing gnb_clf, total=   0.0s

 GridSearchCV(estimator=Pipeline(steps=[('sel_per', SelectPercentile()),
                                       ('ipca', IncrementalPCA()),
                                       ('gnb_clf', GaussianNB())],
                                verbose=True),
             n_jobs=-1,
             param_grid={'sel_per__percentile': [5, 10, 20, 50, 100],
                         'sel_per__score_func': [<function f_classif at 0x000002165103D280>,
                                                 <function chi2 at 0x000002165103D5E0>,
                                                 functools.partial(<function mutual_info_classif at 0x0000021651363820>, random_state=42)]},
             scoring='recall_weighted', verbose=3)

best_score_: 0.8607843137254901

best_params_: {'sel_per__percen

[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    3.4s finished
C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [222 231 235 236 237 238 239 240 241 242 243 245 247 248 249 250 251 252
 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
 271 272 273 274 275 278 279 280] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [8]:
with open('imp_gscvs_dict_1.pkl', 'wb') as file:
    pickle.dump(obj=imp_gscvs_dict, file=file)

In [10]:
### Well, that was silly of me to return my search data in this structure.
### Reformat and expand data into a dataframe.
imp_gscvs_df = pd.DataFrame(columns=['imp_method', 'selector', 'decomp', 'classifier', 'best_score_',
                                     'gscv'])

for imp, gscv_dict in imp_gscvs_dict.items():
    for steps_name, gscv in gscv_dict.items():
        row_dict = {'imp_method': imp}
        
        steps_lst = steps_name.split('_')
        if len(steps_lst) == 5:
            row_dict['selector'] = '_'.join(steps_lst[:2])
            row_dict['decomp'] = steps_lst[2]
            row_dict['classifier'] = '_'.join(steps_lst[3:])
            row_dict['best_score_'] = gscv.best_score_
            row_dict['gscv'] = gscv
        
        row_dict.update(gscv.best_params_)
        imp_gscvs_df = imp_gscvs_df.append(row_dict, ignore_index=True)
        
imp_gscvs_df = imp_gscvs_df.set_index(keys=['imp_method', 'selector', 'decomp', 'classifier'])
imp_gscvs_df.info()
imp_gscvs_df

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 30 entries, ('imp0', 'sel_per', 'fica', 'dt_clf') to ('imp_mv', 'sel_per', 'ipca', 'gnb_clf')
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   best_score_                30 non-null     float64
 1   gscv                       30 non-null     object 
 2   dt_clf__random_state       6 non-null      float64
 3   fica__algorithm            15 non-null     object 
 4   fica__fun                  15 non-null     object 
 5   fica__random_state         15 non-null     float64
 6   sel_per__percentile        30 non-null     float64
 7   sel_per__score_func        30 non-null     object 
 8   rf_clf__bootstrap          6 non-null      float64
 9   rf_clf__max_depth          6 non-null      float64
 10  rf_clf__max_features       6 non-null      object 
 11  rf_clf__min_samples_leaf   6 non-null      float64
 12  rf_clf__min_samples_split  6 non-n

best_score_  \
imp_method selector decomp classifier                
imp0       sel_per  fica   dt_clf         0.824837   
                           rf_clf         0.872549   
                           ab_clf         0.860784   
                           kn_clf         0.884314   
                           gnb_clf        0.860784   
                    ipca   dt_clf         0.732026   
                           rf_clf         0.872549   
                           ab_clf         0.860784   
                           kn_clf         0.872549   
                           gnb_clf        0.860784   
imp_med    sel_per  fica   dt_clf         0.825490   
                           rf_clf         0.883660   
                           ab_clf         0.872549   
                           kn_clf         0.884314   
                           gnb_clf        0.872549   
                    ipca   dt_clf         0.791503   
                           rf_clf         0.883660   
                           ab_clf         0.860784   
                           kn_clf         0.861438   
                           gnb_clf        0.860784   
imp_mv     sel_per  fica   dt_clf         0.815033   
                           rf_clf         0.883660   
                           ab_clf         0.872549   
                           kn_clf         0.872549   
                           gnb_clf        0.862092   
                    ipca   dt_clf         0.814379   
                           rf_clf         0.872549   
                           ab_clf         0.872549   
                           kn_clf         0.872549   
                           gnb_clf        0.860784   

                                                                                    gscv  \
imp_method selector decomp classifier                                                      
imp0       sel_per  fica   dt_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           gnb_clf     GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                    ipca   dt_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           gnb_clf     GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_med    sel_per  fica   dt_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           gnb_clf     GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                    ipca   dt_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           gnb_clf     GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_mv     sel_per  fica   dt_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
          

In [13]:
### Sort by best_score_.
print('Best scores sorted:\n')
imp_gscvs_df.sort_values(by='best_score_', ascending=False)

### Sort by methods and steps.
# print('\nSorted by methods and steps:\n')
# imp_gscvs_df.sort_values(by=['imp_method', 'selector', 'decomp', 'classifier'])

### Groupby methods/steps and compare count and score min, max, mean.
print('\nImputation method stats:\n')
imp_gscvs_df.groupby(by='imp_method').min()
imp_gscvs_df.groupby(by='imp_method')['best_score_'].mean()
imp_gscvs_df.groupby(by='imp_method')['best_score_'].max()

print('\nDecomposition method stats:\n')
imp_gscvs_df.groupby(by='decomp').count()
imp_gscvs_df.groupby(by='decomp')['best_score_'].min()
imp_gscvs_df.groupby(by='decomp')['best_score_'].mean()
imp_gscvs_df.groupby(by='decomp')['best_score_'].max()

print('\nClassifier stats:\n')
imp_gscvs_df.groupby(by='classifier').count()
imp_gscvs_df.groupby(by='classifier')['best_score_'].min()
imp_gscvs_df.groupby(by='classifier')['best_score_'].mean()
imp_gscvs_df.groupby(by='classifier')['best_score_'].max()

###  Groupby selector parameters.
print('\nCount of SelectPercentile best score functions:\n')
imp_gscvs_df.groupby(by='sel_per__score_func').count()
print('\nCount of SelectPercentile best percentiles:\n')
imp_gscvs_df.groupby(by='sel_per__percentile').count()

### Groupby decomp parameters.
print('\nCount of FastICA best algorithms:\n')
imp_gscvs_df.groupby(by='fica__algorithm').count()
print('\nCount of FastICA best functions:\n')
imp_gscvs_df.groupby(by='fica__fun').count()

### Groupby classifier parameters.
for clf, clf_dict in classifiers.items():
    for param in clf['params'].keys():
        print('\nCount of', str(clf_dict['clf']), 'best', param, ':\n')
        imp_gscvs_df.groupby(by=param).count()

Best scores sorted:



best_score_  \
imp_method selector decomp classifier                
imp0       sel_per  fica   kn_clf         0.884314   
imp_med    sel_per  fica   kn_clf         0.884314   
imp_mv     sel_per  fica   rf_clf         0.883660   
imp_med    sel_per  ipca   rf_clf         0.883660   
                    fica   rf_clf         0.883660   
                           ab_clf         0.872549   
imp0       sel_per  fica   rf_clf         0.872549   
imp_mv     sel_per  ipca   kn_clf         0.872549   
imp0       sel_per  ipca   rf_clf         0.872549   
                           kn_clf         0.872549   
imp_mv     sel_per  ipca   rf_clf         0.872549   
imp_med    sel_per  fica   gnb_clf        0.872549   
imp_mv     sel_per  fica   kn_clf         0.872549   
                           ab_clf         0.872549   
                    ipca   ab_clf         0.872549   
                    fica   gnb_clf        0.862092   
imp_med    sel_per  ipca   kn_clf         0.861438   
                           ab_clf         0.860784   
                           gnb_clf        0.860784   
imp0       sel_per  ipca   gnb_clf        0.860784   
                           ab_clf         0.860784   
                    fica   gnb_clf        0.860784   
                           ab_clf         0.860784   
imp_mv     sel_per  ipca   gnb_clf        0.860784   
imp_med    sel_per  fica   dt_clf         0.825490   
imp0       sel_per  fica   dt_clf         0.824837   
imp_mv     sel_per  fica   dt_clf         0.815033   
                    ipca   dt_clf         0.814379   
imp_med    sel_per  ipca   dt_clf         0.791503   
imp0       sel_per  ipca   dt_clf         0.732026   

                                                                                    gscv  \
imp_method selector decomp classifier                                                      
imp0       sel_per  fica   kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_med    sel_per  fica   kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_mv     sel_per  fica   rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_med    sel_per  ipca   rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                    fica   rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp0       sel_per  fica   rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_mv     sel_per  ipca   kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp0       sel_per  ipca   rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_mv     sel_per  ipca   rf_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_med    sel_per  fica   gnb_clf     GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_mv     sel_per  fica   kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                    ipca   ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                    fica   gnb_clf     GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_med    sel_per  ipca   kn_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           gnb_clf     GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp0       sel_per  ipca   gnb_clf     GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                    fica   gnb_clf     GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
                           ab_clf      GridSearchCV(estimator=Pipeline(steps=[('sel_p...   
imp_mv    


Imputation method stats:



,best_score_,gscv,dt_clf__random_state,fica__algorithm,fica__fun,fica__random_state,sel_per__percentile,sel_per__score_func,rf_clf__bootstrap,rf_clf__max_depth,...,rf_clf__random_state,ab_clf__algorithm,ab_clf__base_estimator,ab_clf__n_estimators,ab_clf__random_state,kn_clf__algorithm,kn_clf__leaf_size,kn_clf__n_jobs,kn_clf__n_neighbors,kn_clf__weights
imp_method,,,,,,,,,,,,,,,,,,,,,
imp0,0.732026,NaN,42.0,NaN,NaN,42.0,5.0,NaN,1.0,32.0,...,42.0,NaN,NaN,32.0,42.0,NaN,16.0,-1.0,4.0,NaN
imp_med,0.791503,NaN,42.0,NaN,NaN,42.0,5.0,NaN,0.0,32.0,...,42.0,NaN,NaN,32.0,42.0,NaN,16.0,-1.0,2.0,NaN
imp_mv,0.814379,NaN,42.0,NaN,NaN,42.0,5.0,NaN,1.0,32.0,...,42.0,NaN,NaN,32.0,42.0,NaN,16.0,-1.0,8.0,NaN


imp_method
imp0       0.850196
imp_med    0.859673
imp_mv     0.859869
Name: best_score_, dtype: float64

...

classifier
ab_clf     0.860784
dt_clf     0.732026
gnb_clf    0.860784
kn_clf     0.861438
rf_clf     0.872549
Name: best_score_, dtype: float64

classifier
ab_clf     0.866667
dt_clf     0.800545
gnb_clf    0.862963
kn_clf     0.874619
rf_clf     0.878105
Name: best_score_, dtype: float64

classifier
ab_clf     0.872549
dt_clf     0.825490
gnb_clf    0.872549
kn_clf     0.884314
rf_clf     0.883660
Name: best_score_, dtype: float64


Count of SelectPercentile best score functions:



TypeError: '<' not supported between instances of 'function' and 'function'

In [17]:
# X_train_scaled_imp0.sum()[~np.isfinite(X_train_scaled_imp0.sum())]
# X_train_scaled_imp_med.sum()[~np.isfinite(X_train_scaled_imp_med.sum())]
# X_train_scaled_imp_mv.sum()[~np.isfinite(X_train_scaled_imp_mv.sum())]
# X_train_scaled_imp0.sum().sum()
# X_train_scaled_imp_med.sum().sum()
# X_train_scaled_imp_mv.sum().sum()

Series([], dtype: float64)

Series([], dtype: float64)

Series([], dtype: float64)

13521.008847597379

14188.19415397837

14267.985162541137

In [28]:
### From PCA
# ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

# # https://github.com/scikit-learn/scikit-learn/blob/0.17.X/sklearn/utils/validation.py#L51
    
# # def _assert_all_finite(X):
# #     """Like assert_all_finite, but only for ndarray."""
# #     X = np.asanyarray(X)
# #     # First try an O(n) time, O(1) space solution for the common case that
# #     # everything is finite; fall back to O(n) space np.isfinite to prevent
# #     # false positives from overflow in sum method.
# #     if (X.dtype.char in np.typecodes['AllFloat'] and not np.isfinite(X.sum())
# #             and not np.isfinite(X).all()):
# #         raise ValueError("Input contains NaN, infinity"
# #                          " or a value too large for %r." % X.dtype)

# (X_train_scaled_imp0.values.astype(np.float32).dtype.char in np.typecodes['AllFloat'] \
# and not np.isfinite(X_train_scaled_imp0.values.astype(np.float32).sum()) \
# and not np.isfinite(X_train_scaled_imp0.values.astype(np.float32)).all())

# (X_train_scaled_imp_med.values.astype(np.float32).dtype.char in np.typecodes['AllFloat'] \
# and not np.isfinite(X_train_scaled_imp_med.values.astype(np.float32).sum()) \
# and not np.isfinite(X_train_scaled_imp_med.values.astype(np.float32)).all())

# (X_train_scaled_imp_mv.values.astype(np.float32).dtype.char in np.typecodes['AllFloat'] \
# and not np.isfinite(X_train_scaled_imp_mv.values.astype(np.float32).sum()) \
# and not np.isfinite(X_train_scaled_imp_mv.values.astype(np.float32)).all())


### From my local source code, pointed to in the error message:
# File "C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\utils\validation.py", line 96, in _assert_all_finite
#     raise ValueError(
# ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

# def _assert_all_finite(X, allow_nan=False, msg_dtype=None):
#     """Like assert_all_finite, but only for ndarray."""
#     # validation is also imported in extmath
#     from .extmath import _safe_accumulator_op

#     if _get_config()['assume_finite']:
#         return
#     X = np.asanyarray(X)
#     # First try an O(n) time, O(1) space solution for the common case that
#     # everything is finite; fall back to O(n) space np.isfinite to prevent
#     # false positives from overflow in sum method. The sum is also calculated
#     # safely to reduce dtype induced overflows.
#     is_float = X.dtype.kind in 'fc'
#     if is_float and (np.isfinite(_safe_accumulator_op(np.sum, X))):
#         pass
#     elif is_float:
#         msg_err = "Input contains {} or a value too large for {!r}."
#         if (allow_nan and np.isinf(X).any() or
#                 not allow_nan and not np.isfinite(X).all()):
#             type_err = 'infinity' if allow_nan else 'NaN, infinity'
#             raise ValueError(
#                     msg_err.format
#                     (type_err,
#                      msg_dtype if msg_dtype is not None else X.dtype)
#             )
#     # for object dtype data, we only check for NaNs (GH-13254)
#     elif X.dtype == np.dtype('object') and not allow_nan:
#         if _object_dtype_isnan(X).any():
#             raise ValueError("Input contains NaN")
            
### Based on the error message, this should be the only check:
# (not np.isfinite(X_train_scaled_imp0.values.astype(np.float32)).all())
# (not np.isfinite(X_train_scaled_imp_med.values.astype(np.float32)).all())
# (not np.isfinite(X_train_scaled_imp_mv.values.astype(np.float32)).all())

False

False

False

In [34]:
# pca = PCA(whiten=True, random_state=42)
# pca.fit(X=X_train_scaled_imp0)
# pca.transform(X=X_train_scaled_imp0)
# pca.get_covariance()
# pca.get_params()
# pca.get_precision()
# pca.score(X=X_train_scaled_imp0)
# pca.score_samples(X=X_train_scaled_imp0)

# pca = PCA(whiten=True, random_state=42)
# pca.fit(X=X_train_scaled_imp_med)
# pca.transform(X=X_train_scaled_imp_med)
# pca.get_covariance()
# pca.get_params()
# pca.get_precision()
# pca.score(X=X_train_scaled_imp_med)
# pca.score_samples(X=X_train_scaled_imp_med)

# pca = PCA(whiten=True, random_state=42)
# pca.fit(X=X_train_scaled_imp_mv)
# pca.transform(X=X_train_scaled_imp_mv)
# pca.get_covariance()
# pca.get_params()
# pca.get_precision()
# pca.score(X=X_train_scaled_imp_mv)
# pca.score_samples(X=X_train_scaled_imp_mv)

PCA(random_state=42, whiten=True)

array([[-0.34496174,  1.47870393, -1.22506969, ...,  0.11864948,
        -0.14517229,  1.02260597],
       [ 0.26249544, -0.71134713,  0.44752948, ..., -0.13890838,
         0.52826281,  1.34221523],
       [ 1.39505295,  0.4635651 ,  0.69364513, ..., -0.7063947 ,
        -0.25183749,  1.24778858],
       ...,
       [ 0.48478469, -1.18287214,  0.87100156, ...,  0.68480197,
        -1.04277899,  1.16151135],
       [-1.32249276, -0.83671031, -0.64689213, ...,  0.43891924,
         0.78204994,  0.88042913],
       [-0.07018995, -1.36710711,  1.01669217, ...,  0.55708258,
         0.29203356,  1.01059382]])

array([[ 0.04456265,  0.04988379,  0.04991755, ...,  0.00417373,
         0.00309986,  0.02113268],
       [ 0.04988379,  0.06926293,  0.06165998, ...,  0.00174924,
        -0.00179567,  0.02556269],
       [ 0.04991755,  0.06165998,  0.10504885, ...,  0.00021737,
        -0.00174333,  0.05901886],
       ...,
       [ 0.00417373,  0.00174924,  0.00021737, ...,  0.11735653,
         0.11437824,  0.04471577],
       [ 0.00309986, -0.00179567, -0.00174333, ...,  0.11437824,
         0.11379552,  0.04096007],
       [ 0.02113268,  0.02556269,  0.05901886, ...,  0.04471577,
         0.04096007,  0.0729415 ]])

{'copy': True,
 'iterated_power': 'auto',
 'n_components': None,
 'random_state': 42,
 'svd_solver': 'auto',
 'tol': 0.0,
 'whiten': True}

C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\decomposition\_base.py:72: RuntimeWarning: divide by zero encountered in true_divide
  precision = np.dot(components_, components_.T) / self.noise_variance_


ValueError: array must not contain infs or NaNs

In [39]:
### from sklearn.decomposition.PCA.get_precision()

# ~\anaconda3\envs\py3\lib\site-packages\numpy\lib\function_base.py in asarray_chkfinite(a, dtype, order)
#     483     a = asarray(a, dtype=dtype, order=order)
#     484     if a.dtype.char in typecodes['AllFloat'] and not np.isfinite(a).all():
# --> 485         raise ValueError(
#     486             "array must not contain infs or NaNs")
#     487     return a

# ValueError: array must not contain infs or NaNs

# def asarray_chkfinite(a, dtype=None, order=None):
#     """Convert the input to an array, checking for NaNs or Infs.

#     Parameters
#     ----------
#     a : array_like
#         Input data, in any form that can be converted to an array.  This
#         includes lists, lists of tuples, tuples, tuples of tuples, tuples
#         of lists and ndarrays.  Success requires no NaNs or Infs.
#     dtype : data-type, optional
#         By default, the data-type is inferred from the input data.
#     order : {'C', 'F'}, optional
#          Whether to use row-major (C-style) or
#          column-major (Fortran-style) memory representation.
#          Defaults to 'C'.

#     Returns
#     -------
#     out : ndarray
#         Array interpretation of `a`.  No copy is performed if the input
#         is already an ndarray.  If `a` is a subclass of ndarray, a base
#         class ndarray is returned.

#     Raises
#     ------
#     ValueError
#         Raises ValueError if `a` contains NaN (Not a Number) or Inf (Infinity).

#     See Also
#     --------
#     asarray : Create and array.
#     asanyarray : Similar function which passes through subclasses.
#     ascontiguousarray : Convert input to a contiguous array.
#     asfarray : Convert input to a floating point ndarray.
#     asfortranarray : Convert input to an ndarray with column-major
#                      memory order.
#     fromiter : Create an array from an iterator.
#     fromfunction : Construct an array by executing a function on grid
#                    positions.

#     Examples
#     --------
#     Convert a list into an array.  If all elements are finite
#     ``asarray_chkfinite`` is identical to ``asarray``.

#     >>> a = [1, 2]
#     >>> np.asarray_chkfinite(a, dtype=float)
#     array([1., 2.])

#     Raises ValueError if array_like contains Nans or Infs.

#     >>> a = [1, 2, np.inf]
#     >>> try:
#     ...     np.asarray_chkfinite(a)
#     ... except ValueError:
#     ...     print('ValueError')
#     ...
#     ValueError

#     """
#     a = asarray(a, dtype=dtype, order=order)
#     if a.dtype.char in typecodes['AllFloat'] and not np.isfinite(a).all():
#         raise ValueError(
#             "array must not contain infs or NaNs")
#     return a

### Error check:

### From numpy.numerictypes
# typecodes = {'Character':'c',
#              'Integer':'bhilqp',
#              'UnsignedInteger':'BHILQP',
#              'Float':'efdg',
#              'Complex':'FDG',
#              'AllInteger':'bBhHiIlLqQpP',
#              'AllFloat':'efdgFDG',
#              'Datetime': 'Mm',
#              'All':'?bhilqpBHILQPefdgFDGSUVOMm'}
# ###

# a = X_train_scaled_imp0.values
# a.dtype.char in typecodes['AllFloat'] and not np.isfinite(a).all()

# a = X_train_scaled_imp_med
# a.dtype.char in typecodes['AllFloat'] and not np.isfinite(a).all()

# a = X_train_scaled_imp_mv
# a.dtype.char in typecodes['AllFloat'] and not np.isfinite(a).all()

False

False

False

In [13]:
# X_train_scaled_imp0.values.astype(np.float32)
# np.isnan(X_train_scaled_imp0.values.astype(np.float32)).sum()
# np.isinf(X_train_scaled_imp0.values.astype(np.float32)).sum()

# X_train_scaled_imp_med.values.astype(np.float32)
# np.isnan(X_train_scaled_imp_med.values.astype(np.float32)).sum()
# np.isinf(X_train_scaled_imp_med.values.astype(np.float32)).sum()

# X_train_scaled_imp_mv.values.astype(np.float32)
# np.isnan(X_train_scaled_imp_mv.values.astype(np.float32)).sum()
# np.isinf(X_train_scaled_imp_mv.values.astype(np.float32)).sum()

array([[0.1022806 , 0.02350814, 1.        , ..., 0.42857143, 0.375     ,
        0.8888889 ],
       [0.19200876, 0.0596745 , 0.13653117, ..., 0.14285715, 0.125     ,
        0.22222222],
       [0.19539087, 0.23146474, 0.4885242 , ..., 0.85714287, 0.75      ,
        0.7777778 ],
       ...,
       [0.16596174, 0.09584087, 0.18990771, ..., 0.2857143 , 0.25      ,
        0.22222222],
       [0.        , 0.        , 0.        , ..., 0.71428573, 0.625     ,
        0.5555556 ],
       [0.1477351 , 0.0596745 , 0.0291929 , ..., 0.2857143 , 0.25      ,
        0.22222222]], dtype=float32)

0

0

array([[0.1022806 , 0.02350814, 1.        , ..., 0.2857143 , 0.125     ,
        0.11111111],
       [0.19200876, 0.0596745 , 0.13653117, ..., 0.85714287, 0.25      ,
        0.22222222],
       [0.19539087, 0.23146474, 0.4885242 , ..., 0.        , 0.75      ,
        0.6666667 ],
       ...,
       [0.16596174, 0.09584087, 0.18990771, ..., 0.5714286 , 0.25      ,
        0.22222222],
       [0.17216934, 0.12296564, 0.20065798, ..., 0.14285715, 0.875     ,
        0.7777778 ],
       [0.1477351 , 0.0596745 , 0.0291929 , ..., 0.5714286 , 0.25      ,
        0.22222222]], dtype=float32)

0

0

array([[0.1022806 , 0.02350814, 1.        , ..., 0.14285715, 0.25      ,
        0.22222222],
       [0.19200876, 0.0596745 , 0.13653117, ..., 0.2857143 , 0.75      ,
        0.6666667 ],
       [0.19539087, 0.23146474, 0.4885242 , ..., 0.85714287, 0.        ,
        0.        ],
       ...,
       [0.16596174, 0.09584087, 0.18990771, ..., 0.2857143 , 0.875     ,
        0.7777778 ],
       [0.08125323, 0.08637901, 0.23989324, ..., 1.        , 0.125     ,
        0.11111111],
       [0.1477351 , 0.0596745 , 0.0291929 , ..., 0.2857143 , 0.875     ,
        0.7777778 ]], dtype=float32)

0

0